In [14]:
!unzip chest-xray-pneumonia.zip

Archive:  chest-xray-pneumonia.zip
  inflating: chest_xray.zip          


In [0]:
!ls -al
!unzip chest_xray.zip

In [1]:
import os
from keras.preprocessing.image import ImageDataGenerator
from keras.backend import clear_session
from keras.optimizers import SGD
from pathlib import Path
from keras.applications import InceptionV3
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Dropout, Flatten, AveragePooling2D
from keras import initializers, regularizers

Using TensorFlow backend.


In [0]:
# Config
height = 299
width = height

In [4]:
conv_base = InceptionV3(
    weights='imagenet', 
    include_top=False, 
    input_shape=(height, width, 3)
)

# First time run, no unlocking
conv_base.trainable = False









87916544/87910968 [==============================] - 29s 0us/step


In [5]:
# Let's see it
print('Summary')
print(conv_base.summary())

Summary
Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 149, 149, 32) 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 149, 149, 32) 0           batch_normalization_1[0][0]      
_______________________________________________________________________________

In [8]:
num_classes = 2
# Let's construct that top layer replacement
x = conv_base.output
x = AveragePooling2D(pool_size=(8, 8))(x)
x - Dropout(0.4)(x)
x = Flatten()(x)
x = Dense(256, activation='relu', kernel_initializer=initializers.he_normal(seed=None), kernel_regularizer=regularizers.l2(.0005))(x)
x = Dropout(0.5)(x)
# Essential to have another layer for better accuracy
x = Dense(128,activation='relu', kernel_initializer=initializers.he_normal(seed=None))(x)
x = Dropout(0.25)(x)
predictions = Dense(num_classes,  kernel_initializer="glorot_uniform", activation='softmax')(x)

print('Stacking New Layers')
model = Model(inputs = conv_base.input, outputs=predictions)

Stacking New Layers


In [0]:
from keras.callbacks import LearningRateScheduler

# Slow down training deeper into dataset
def schedule(epoch):
    if epoch < 6:
        # Warmup model first
        return .0000032
    elif epoch < 12:
        return .01
    elif epoch < 15:
        return .002
    elif epoch < 18:
        return .0004
    elif epoch < 20:
        return .00008
    elif epoch < 80:
        return .000016
    elif epoch < 95:
        return .0000032        
    else:
        return .0000009    
      
lr_scheduler = LearningRateScheduler(schedule)


In [10]:
print('Compile model')
opt = SGD(momentum=.9)
model.compile(
    loss='categorical_crossentropy',
    optimizer=opt,
    metrics=['accuracy']
)

Compile model



In [0]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    channel_shift_range=20,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Validation data should not be modified
validation_datagen = ImageDataGenerator(
    rescale=1./255
)

In [0]:
train_dir = os.path.join('./chest_xray', 'train')
test_dir = os.path.join('./chest_xray', 'test')

In [17]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(height, width),
    class_mode='categorical',
    batch_size=64
)

validation_generator = validation_datagen.flow_from_directory(
    test_dir,
    target_size=(height, width),
    class_mode='categorical',
    batch_size=64
)

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [18]:
history = model.fit_generator(
    train_generator,
    callbacks=[lr_scheduler],
    epochs=20,
    steps_per_epoch=50,
    shuffle=True,
    workers=4,
    use_multiprocessing=False,
    validation_data=validation_generator,
    validation_steps=10
)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/20
50/50 [==============================] - 122s 2s/step - loss: 0.9498 - acc: 0.5831 - val_loss: 0.9112 - val_acc: 0.6330
Epoch 2/20
50/50 [==============================] - 77s 2s/step - loss: 0.9439 - acc: 0.6050 - val_loss: 0.9074 - val_acc: 0.6170
Epoch 3/20
50/50 [==============================] - 80s 2s/step - loss: 0.9308 - acc: 0.6156 - val_loss: 0.9062 - val_acc: 0.6346
Epoch 4/20
50/50 [==============================] - 81s 2s/step - loss: 0.9348 - acc: 0.6162 - val_loss: 0.9082 - val_acc: 0.6234
Epoch 5/20
50/50 [==============================] - 83s 2s/step - loss: 0.9256 - acc: 0.6253 - val_loss: 0.9082 - val_acc: 0.6362
Epoch 6/20
50/50 [==============================] - 83s 2s/step - loss: 0.9110 - acc: 0.6428 - val_loss: 0.9041 - val_acc: 0.6362
Epoch 7/20
50/50 [==============================] - 82s 2s/step - loss: 0.5301 - acc: 0.8809 - val_loss: 0.9311 - val_acc: 0.

In [19]:
# Save it for later
print('Saving Model')
model.save("xray.h5")

Saving Model


# Now demo the model on the whole validation set

In [0]:
from keras.preprocessing import image
import numpy as np

In [29]:
for f in sorted(Path("./chest_xray/val/").glob("**/*.jpeg")):
    # Load an image file to test
    image_to_test = image.load_img(
        str(f),
        target_size=(299, 299)
    )

    # Convert the image data to a numpy array
    # suitable for Keras
    image_to_test = image.img_to_array(image_to_test)
    # normalize to a 0 to 1 value
    image_to_test /= 255

    # Add a fourth dimension to the image since
    # Keras expects a list of images
    list_of_images = np.expand_dims(
        image_to_test,
        axis=0
    )
    # Make a prediction using the model
    results = model.predict(list_of_images)

    # Since we only passed in one test image,
    # we can just check the first result directly.
    image_likelihood = results[0][0]

    # The result will be a number from 0.0 to 1.0
    # representing the likelihood that this
    # image is a truck.
    if image_likelihood > 0.4:
        print(f"{f} is a NORMAL! ({image_likelihood:.2f})")
    else:
        print(f"{f} is pneumonia! ({image_likelihood:.2f})")

chest_xray/val/NORMAL/NORMAL2-IM-1427-0001.jpeg is a NORMAL! (0.87)
chest_xray/val/NORMAL/NORMAL2-IM-1430-0001.jpeg is a NORMAL! (0.99)
chest_xray/val/NORMAL/NORMAL2-IM-1431-0001.jpeg is a NORMAL! (0.70)
chest_xray/val/NORMAL/NORMAL2-IM-1436-0001.jpeg is a NORMAL! (0.99)
chest_xray/val/NORMAL/NORMAL2-IM-1437-0001.jpeg is a NORMAL! (0.46)
chest_xray/val/NORMAL/NORMAL2-IM-1438-0001.jpeg is pneumonia! (0.15)
chest_xray/val/NORMAL/NORMAL2-IM-1440-0001.jpeg is a NORMAL! (0.97)
chest_xray/val/NORMAL/NORMAL2-IM-1442-0001.jpeg is a NORMAL! (1.00)
chest_xray/val/PNEUMONIA/person1946_bacteria_4874.jpeg is pneumonia! (0.00)
chest_xray/val/PNEUMONIA/person1946_bacteria_4875.jpeg is pneumonia! (0.00)
chest_xray/val/PNEUMONIA/person1947_bacteria_4876.jpeg is pneumonia! (0.20)
chest_xray/val/PNEUMONIA/person1949_bacteria_4880.jpeg is pneumonia! (0.00)
chest_xray/val/PNEUMONIA/person1950_bacteria_4881.jpeg is pneumonia! (0.01)
chest_xray/val/PNEUMONIA/person1951_bacteria_4882.jpeg is pneumonia! (0.02)